# Two - Tower Retreival Model

### Key resources:
* Many pages [here](https://www.tensorflow.org/recommenders/examples/deep_recommenders) include great techniques to build custom TFRS Models

### Goals:
* Show how to model off of most data types 
  * (String, Existing Embeddings (vectors), 
  * Floats (Normalized), 
  * Categorical with vocab, 
  * High Dim Categorical (Embed)
* Leverage class templates to create custom 2 Tower Models quick/easy

## SPOTIFY Create the tensorflow.io interface for the event and product table in Bigquery
Best practices from Google are in this blog post

In [1]:
# set variables
DROPOUT = False
DROPOUT_RATE = 0.2
EMBEDDING_DIM = 64
MAX_TOKENS = 100_000
BATCH_SIZE = 256
ARCH = [128, 64]
NUM_EPOCHS = 1
SEED = 41781897
PROJECT_ID = 'jtotten-project'
DROP_FIELDS = ['pid', 'track_uri', 'artist_uri', 'album_uri']
N_RECORDS_PER_TFRECORD_FILE = 300 * 50 #100ish mb  
TF_RECORDS_DIR = 'gs://spotify-tfrecords'

#### Quick counts on training data



#### Quick counts on the training records for track

In [2]:
%%bigquery TOTAL_PLAYLISTS
select count(1) from jtotten-project.spotify_mpd.playlists_track_string

Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.34rows/s]


In [3]:
TOTAL_PLAYLISTS = TOTAL_PLAYLISTS.values[0][0]
TOTAL_PLAYLISTS

1032000

#### Same with playlist

#### Quick counts (this time playlists) on the training records for track

In [4]:
%%bigquery TOTAL_TRACKS
select count(1) from jtotten-project.spotify_mpd.track_audio

Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.30rows/s]


In [5]:
TOTAL_TRACKS = TOTAL_TRACKS.values[0][0]
TOTAL_TRACKS

2261490

### Set the tf.io pipelines function from bigquery

[Great blog post here on it](https://towardsdatascience.com/how-to-read-bigquery-data-from-tensorflow-2-0-efficiently-9234b69165c8)

In [6]:
import tensorflow as tf
from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
from tensorflow_io.bigquery import BigQueryReadSession
import warnings
warnings.filterwarnings("ignore") #do this b/c there's an info-level bug that can safely be ignored
import json
import tensorflow as tf
import tensorflow_recommenders as tfrs
import datetime
from tensorflow.python.lib.io import file_io
from tensorflow.train import BytesList, Feature, FeatureList, Int64List
from tensorflow.train import SequenceExample, FeatureLists



def bq_to_tfdata(client, row_restriction, table_id, col_names, col_types, dataset, batch_size=BATCH_SIZE):
    TABLE_ID = table_id
    COL_NAMES = col_names
    COL_TYPES = col_types
    DATASET = dataset
    bqsession = client.read_session(
        "projects/" + PROJECT_ID,
        PROJECT_ID, TABLE_ID, DATASET,
        COL_NAMES, COL_TYPES,
        requested_streams=2,
        row_restriction=row_restriction)
    dataset = bqsession.parallel_read_rows()
    return dataset.prefetch(1).shuffle(batch_size*10).batch(batch_size)

## Get the song metadata

To get a pipeline working we need the metadata for the table along with the table information. The following functions are helpers that give us the metadata into the proper types for `tf`


For each table id, programatically get
* Column names
* Column types

In [7]:
%%bigquery schema
SELECT * FROM jtotten-project.spotify_mpd.INFORMATION_SCHEMA.TABLES
where table_name in ('track_audio', 'playlists_track_string');

Downloading: 100%|██████████| 2/2 [00:00<00:00,  4.07rows/s]


In [8]:
schema # we will get the fields out of the ddl field

,table_catalog,table_schema,table_name,table_type,is_insertable_into,is_typed,creation_time,base_table_catalog,base_table_schema,base_table_name,snapshot_time_ms,ddl
0,jtotten-project,spotify_mpd,track_audio,BASE TABLE,YES,NO,2022-04-06 17:46:25.801000+00:00,None,None,None,NaT,CREATE TABLE `jtotten-project.spotify_mpd.trac...
1,jtotten-project,spotify_mpd,playlists_track_string,BASE TABLE,YES,NO,2022-04-22 22:50:46.601000+00:00,None,None,None,NaT,CREATE TABLE `jtotten-project.spotify_mpd.play...


## Helper functions to pull metadata from ddl statements

In [9]:
# Function to convert string type representation to tf data types

def conv_dtype_to_tf(dtype_str):
    if dtype_str == 'FLOAT64':
        return dtypes.float64
    elif dtype_str == 'INT64':
        return dtypes.int64
    else: 
        return dtypes.string
        
def get_metadata_from_ddl(ddl, drop_field=None):
    fields = []
    types = []
    ddl = ddl.values[0]
    for line in ddl.splitlines():
        if line[:1] == ' ': #only pull indented lines for the fields
            # drop the comma
            line = line.replace(',','')
            space_delim = line.split(' ')
            if space_delim[2] in drop_field:
                pass
            else:
                fields.append(space_delim[2])
                types.append(conv_dtype_to_tf(space_delim[3]))
    return fields, types


track_audio_fields, track_audio_types = get_metadata_from_ddl(schema.ddl[schema.table_name == 'track_audio'], DROP_FIELDS)
playlist_fields, playlist_types = get_metadata_from_ddl(schema.ddl[schema.table_name == 'playlists_track_string'], DROP_FIELDS) 

In [10]:
# Quick check on data
for a, b in zip(playlist_fields, playlist_types):
    print(a +" : " + str(b))

name : <dtype: 'string'>
collaborative : <dtype: 'string'>
modified_at : <dtype: 'int64'>
num_tracks : <dtype: 'int64'>
num_albums : <dtype: 'int64'>
num_followers : <dtype: 'int64'>
tracks : <dtype: 'string'>
num_edits : <dtype: 'int64'>
duration_ms : <dtype: 'int64'>
num_artists : <dtype: 'int64'>
description : <dtype: 'string'>


In [11]:
# Quick check on data
for a, b in zip(track_audio_fields, track_audio_types):
    print(a +" : " + str(b))
    
DROP_TRACK_AUDIO_FIELDS = ['pid', 'track_uri', 'artist_uri', 'album_uri']

artist_name : <dtype: 'string'>
track_name : <dtype: 'string'>
album_name : <dtype: 'string'>
name : <dtype: 'string'>
danceability : <dtype: 'float64'>
energy : <dtype: 'float64'>
key : <dtype: 'float64'>
loudness : <dtype: 'float64'>
mode : <dtype: 'float64'>
speechiness : <dtype: 'float64'>
acousticness : <dtype: 'float64'>
instrumentalness : <dtype: 'float64'>
liveness : <dtype: 'float64'>
valence : <dtype: 'float64'>
tempo : <dtype: 'float64'>
type : <dtype: 'string'>
id : <dtype: 'string'>
uri : <dtype: 'string'>
track_href : <dtype: 'string'>
analysis_url : <dtype: 'string'>
time_signature : <dtype: 'float64'>
artist_pop : <dtype: 'int64'>
track_pop : <dtype: 'string'>
genres : <dtype: 'string'>
duration_ms : <dtype: 'int64'>


### Now the helper functions are set. Below tf.data pipelines are created from bigquery

In [12]:
track_train_pipeline = bq_to_tfdata(BigQueryClient(), row_restriction=None, table_id = 'track_audio'
                                    , col_names=track_audio_fields, col_types=track_audio_types, dataset='spotify_mpd', batch_size=1) #we will change to BATCH_SIZE after we test 

2022-04-24 20:02:37.141644: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available
2022-04-24 20:02:37.142160: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: AVX2 FMA
2022-04-24 20:02:38.514837: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-24 20:02:38.515586: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-24 20:02:38.525382: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-24 20:02:38.526144: 

In [13]:
### Validate we are getting records

for line in track_train_pipeline.take(1):
    print(line) #should come out based on batch size

2022-04-24 20:02:40.307367: E tensorflow/core/framework/dataset.cc:577] UNIMPLEMENTED: Cannot compute input sources for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.
2022-04-24 20:02:40.307412: E tensorflow/core/framework/dataset.cc:581] UNIMPLEMENTED: Cannot merge options for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.
2022-04-24 20:02:40.307824: E tensorflow/core/framework/dataset.cc:577] UNIMPLEMENTED: Cannot compute input sources for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.
2022-04-24 20:02:40.307846: E tensorflow/core/framework/dataset.cc:581] UNIMPLEMENTED: Cannot merge options for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.


OrderedDict([('acousticness', <tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.0288])>), ('album_name', <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Dim Mak Records New Noise, Vol. 3'], dtype=object)>), ('analysis_url', <tf.Tensor: shape=(1,), dtype=string, numpy=
array([b'https://api.spotify.com/v1/audio-analysis/6L16Zmb3Wtk1jwj1vuWne4'],
      dtype=object)>), ('artist_name', <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Comic Strips'], dtype=object)>), ('artist_pop', <tf.Tensor: shape=(1,), dtype=int64, numpy=array([-1])>), ('danceability', <tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.83])>), ('duration_ms', <tf.Tensor: shape=(1,), dtype=int64, numpy=array([288634])>), ('energy', <tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.513])>), ('genres', <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'unknown'], dtype=object)>), ('id', <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'6L16Zmb3Wtk1jwj1vuWne4'], dtype=object)>), ('instrumentalness', 

### For the song audio data, we are set and will use this pipeline in training - there's no need to pre-process as there are no nested elements

In [14]:
playlist_types[-1] = dtypes.string #try manually setting the dtype for the tracks nested column

In [15]:
## Validate playlist data
playlist_train_pipeline = bq_to_tfdata(BigQueryClient(), row_restriction=None, table_id = 'playlists_track_string'
                                    , col_names=playlist_fields
                                       , col_types=playlist_types
                                       , dataset='spotify_mpd', batch_size=1)
for line in playlist_train_pipeline.take(1):
    print(line) #should come out based on batch size

2022-04-24 20:02:41.482012: E tensorflow/core/framework/dataset.cc:577] UNIMPLEMENTED: Cannot compute input sources for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.
2022-04-24 20:02:41.482069: E tensorflow/core/framework/dataset.cc:581] UNIMPLEMENTED: Cannot merge options for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.
2022-04-24 20:02:41.482422: E tensorflow/core/framework/dataset.cc:577] UNIMPLEMENTED: Cannot compute input sources for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.
2022-04-24 20:02:41.482462: E tensorflow/core/framework/dataset.cc:581] UNIMPLEMENTED: Cannot merge options for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.


OrderedDict([('collaborative', <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'false'], dtype=object)>), ('description', <tf.Tensor: shape=(1,), dtype=string, numpy=array([b''], dtype=object)>), ('duration_ms', <tf.Tensor: shape=(1,), dtype=int64, numpy=array([4203113])>), ('modified_at', <tf.Tensor: shape=(1,), dtype=int64, numpy=array([1499990400])>), ('name', <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Span'], dtype=object)>), ('num_albums', <tf.Tensor: shape=(1,), dtype=int64, numpy=array([10])>), ('num_artists', <tf.Tensor: shape=(1,), dtype=int64, numpy=array([6])>), ('num_edits', <tf.Tensor: shape=(1,), dtype=int64, numpy=array([3])>), ('num_followers', <tf.Tensor: shape=(1,), dtype=int64, numpy=array([1])>), ('num_tracks', <tf.Tensor: shape=(1,), dtype=int64, numpy=array([18])>), ('tracks', <tf.Tensor: shape=(1,), dtype=string, numpy=
array([b"[{'pos': 0, 'artist_name': 'Carlos Vives', 'track_uri': 'spotify:track:1KbX1VuFIgsRa9JjnQtRYA', 'artist_uri': 'spotify:art

## In pulling one record it looks like it's properly parsing a tf record

# do some data wranglging on the text data
# tf.train.Example(features=tf.train.Features(feature=feature))
for _ in playlist_train_pipeline.map(lambda x: tf.io.parse_sequence_example(tf.io.serialize_tensor(x['tracks'][0]), sequence_features=feature_description, context_features=context_features, name='tracks')).take(1):
    tensor = _
    print(_)

Since the data is stored in a text dictionary we will eagerly execute, grab the values and do a string `eval`
`eval("{'pos': 0, 'artist_name': 'King Crimson', 'track_uri': 'spotify:track:173gp7NIXqk0MEo8K7Av4a', 'artist_uri': 'spotify:artist:7M1FPw29m5FbicYzS2xdpi', 'track_name': '21st Century Schizoid Man', 'album_uri': 'spotify:album:0ga8Q4tTXaFf9q3LvT8hrC', 'duration_ms': 657517, 'album_name': 'Radical Action To Unseat the Hold of Monkey Mind (Live)'}")`

## This funcion parses the playlist data and breaks down the nested fields to be conformant with `SequenceExample`
The 'flat' features come along as `context_features` in `SequenceExample`
There is one more helper function to parse the example and write it to the destination `gs://` path

In [16]:
@tf.function
def get_tensor_from_tracks(tensor):
    key_list = ['pos', 'artist_name', 'track_uri', 'artist_uri', 'track_name', 'album_uri', 'duration_ms', 'album_name']
    y = {}
    
    
    tracks = tensor["tracks"][0]
    tracks = tracks.numpy()

    tracks = eval(tracks)

    for _ in key_list:
        y[_] = []

    for track in tracks:
        y['pos'].append(track['pos'])
        y['artist_name'].append(track['artist_name'].encode('utf8'))
        y['artist_uri'].append(track['artist_uri'].encode('utf8'))
        y['track_name'].append(track['track_name'].encode('utf8'))
        y['album_uri'].append(track['album_uri'].encode('utf8'))
        y['duration_ms'].append(track['duration_ms'])
        y['album_name'].append(track['album_name'].encode('utf8'))
        y['track_uri'].append(track['track_uri'].encode('utf8'))
        


    # set list types
    pos = Int64List(value=y['pos'])
    artist_name = BytesList(value=y['artist_name'])
    artist_uri = BytesList(value=y['artist_uri'])
    track_name = BytesList(value=y['track_name'])
    album_uri = BytesList(value=y['album_uri'])
    duration_ms = Int64List(value=y['duration_ms'])
    album_name = BytesList(value=y['album_name'])
    track_uri = BytesList(value=y['track_uri'])

    
    sample_dict = {
    "name" : Feature(bytes_list=BytesList(value=tensor['name'].numpy())),
    "collaborative" : Feature(bytes_list=BytesList(value=tensor['collaborative'].numpy())),
    "modified_at" : Feature(int64_list=Int64List(value=tensor['modified_at'].numpy())),
    "num_tracks" : Feature(int64_list=Int64List(value=tensor['num_tracks'].numpy())),
    "num_albums" : Feature(int64_list=Int64List(value=tensor['num_albums'].numpy())),
    "num_followers" : Feature(int64_list=Int64List(value=tensor['num_followers'].numpy())),
    "num_edits" : Feature(int64_list=Int64List(value=tensor['num_edits'].numpy())),
    "duration_ms" : Feature(int64_list=Int64List(value=tensor['duration_ms'].numpy())),
    "num_artists" : Feature(int64_list=Int64List(value=tensor['num_artists'].numpy())),
    "description" : Feature(bytes_list=BytesList(value=tensor['description'].numpy()))
    }

    # combine feature list

    fl = FeatureList(feature=[Feature(int64_list=pos), 
                              Feature(bytes_list=artist_name),
                              Feature(bytes_list=artist_uri),
                              Feature(bytes_list=track_name),
                              Feature(bytes_list=album_uri),
                              Feature(int64_list=duration_ms),
                              Feature(bytes_list=album_name),
                              Feature(bytes_list=track_uri)
                             ])

    #create the sequence
    seq = SequenceExample(context=tf.train.Features(feature=sample_dict),
                          feature_lists=FeatureLists(feature_list={
        "tracks": fl
    }))
    
    return seq


def write_a_tfrec(lns, n_records_per_file, file_counter, subfolder):
    #next write to a tfrecord
    with tf.io.TFRecordWriter(
        TF_RECORDS_DIR + "/" + subfolder +"/file_%.2i-%i.tfrec" % (n_records_per_file, file_counter)
    ) as writer:
        for example in lns:
            writer.write(example.SerializeToString())
            
            


## Now iterate over the pipeline
Creating files with batches of `N_RECORDS_PER_TFRECORD_FILE`

In [ ]:
tf.config.run_functions_eagerly(True)
# using datetime module

# ct stores current time
ct = str(datetime.datetime.now()).replace(" ",":")

records = []

for i, line in enumerate(playlist_train_pipeline):
    sequence_example = get_tensor_from_tracks(line) #should come out based on batch size
    file_count = i // N_RECORDS_PER_TFRECORD_FILE ## iterates when floor division increments for i over NRPTF
    if i % N_RECORDS_PER_TFRECORD_FILE == 0 or i == TOTAL_PLAYLISTS-1: #write-a-file and reset the batch
        write_a_tfrec(records, n_records_per_file=N_RECORDS_PER_TFRECORD_FILE, subfolder=ct, file_counter = file_count)
        records = []
    else:
        records.append(sequence_example)

2022-04-24 20:02:41.862578: E tensorflow/core/framework/dataset.cc:577] UNIMPLEMENTED: Cannot compute input sources for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.
2022-04-24 20:02:41.862619: E tensorflow/core/framework/dataset.cc:581] UNIMPLEMENTED: Cannot merge options for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.
2022-04-24 20:02:41.863001: E tensorflow/core/framework/dataset.cc:577] UNIMPLEMENTED: Cannot compute input sources for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.
2022-04-24 20:02:41.863051: E tensorflow/core/framework/dataset.cc:581] UNIMPLEMENTED: Cannot merge options for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.


# Model Draft Stuff

In [13]:
# %%writefile -a vertex_train/trainer/task.py

class PlaylistsModel(tf.keras.Model):
    def __init__(self, layer_sizes, adapt_data):
        super().__init__()
        
        #start with lookups on low cardnality categorical items
        colab_vocab = tf.constant(['true','false'], name='colab_vocab', dtype='string')
        
        self.colab = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=colab_vocab, mask_token=None, name="colab_lookup", output_mode='count')
        ], name="colab")
        
        #create text vectorizors to be fed to an embedding layer
        self.artist_vectorizor = tf.keras.layers.TextVectorization(
            max_tokens=MAX_TOKENS, name="artist_tv", ngrams=2)
        
        self.album_vectorizor = tf.keras.layers.TextVectorization(
            max_tokens=MAX_TOKENS, name="album_tv", ngrams=2)
        
        self.description_vectorizor = tf.keras.layers.TextVectorization(
            max_tokens=MAX_TOKENS, name="album_tv", ngrams=2)
        
        self.query_embedding = tf.keras.Sequential([
            self.album_vectorizor,
            tf.keras.layers.Embedding(MAX_TOKENS+1, EMBEDDING_DIM , mask_zero=True, name="album_emb"),
            tf.keras.layers.GlobalAveragePooling1D()
        ], name="album_embedding_model")
        
        self.artist_embedding = tf.keras.Sequential([
            self.artist_vectorizor,
            tf.keras.layers.Embedding(MAX_TOKENS+1, EMBEDDING_DIM , mask_zero=True, name="artist_emb"),
            tf.keras.layers.GlobalAveragePooling1D()
        ], name="artist_embedding")
        
        ###############
        ### adapt stuff
        ###############
        
        self.artist_vectorizor.adapt(adapt_data.map(lambda x: x['artist_name']))
        self.album_vectorizor.adapt(adapt_data.map(lambda x: x['album_name'])) 
        
        # Then construct the layers.
        self.dense_layers = tf.keras.Sequential(name="dense_layers_query")
        
        initializer = tf.keras.initializers.GlorotUniform(seed=SEED)
        # Use the ReLU activation for all but the last layer.
        for layer_size in layer_sizes[:-1]:
            self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu", kernel_initializer=initializer))
            if DROPOUT:
                self.dense_layers.add(tf.keras.layers.Dropout(DROPOUT_RATE))
        # No activation for the last layer
        for layer_size in layer_sizes[-1:]:
            self.dense_layers.add(tf.keras.layers.Dense(layer_size, kernel_initializer=initializer))
        ### ADDING L2 NORM AT THE END
        self.dense_layers.add(tf.keras.layers.Lambda(lambda x: tf.nn.l2_normalize(x, 1, epsilon=1e-12, name="normalize_dense")))


    def call(self, data):    
        all_embs = tf.concat(
                [
                    self.album_embedding(data['album_name']),
                    self.artist_embedding(data['artist_name']),
                    self.colab(data['collaborative']),
                    self.description_embedding(data['description'])
                ], axis=1)
        return self.dense_layers(all_embs)

## Use the example output to think of how you process your features

```
OrderedDict([('album_name', <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'The Helm'], dtype=object)>), ('artist_name', <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Carrot Green'], dtype=object)>), ('collaborative', <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'false'], dtype=object)>), ('description', <tf.Tensor: shape=(1,), dtype=string, numpy=array([b''], dtype=object)>), ('duration_ms', <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'358500'], dtype=object)>), ('modified_at', <tf.Tensor: shape=(1,), dtype=int64, numpy=array([1505692800])>), ('name', <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'FeSTa'], dtype=object)>), ('num_albums', <tf.Tensor: shape=(1,), dtype=int64, numpy=array([82])>), ('num_artists', <tf.Tensor: shape=(1,), dtype=int64, numpy=array([66])>), ('num_edits', <tf.Tensor: shape=(1,), dtype=int64, numpy=array([48])>), ('num_followers', <tf.Tensor: shape=(1,), dtype=int64, numpy=array([1])>), ('num_tracks', <tf.Tensor: shape=(1,), dtype=int64, numpy=array([85])>), ('pos', <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'45'], dtype=object)>), ('track_name', <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'The Helm - Carrot Green Remix'], dtype=object)>)])
```

In [14]:
#### Tests

In [415]:
# data[0].keys()#originally got the values from this
feature_description = {'pos': tf.io.RaggedFeature(tf.int64), 
                     'artist_name':  tf.io.RaggedFeature(tf.string), 
                     'track_uri':  tf.io.RaggedFeature(tf.string), 
                     'artist_uri': tf.io.RaggedFeature(tf.string), 
                     'track_name': tf.io.RaggedFeature(tf.string), 
                     'album_uri': tf.io.RaggedFeature(tf.string),
                     'duration_ms': tf.io.RaggedFeature(tf.int64), 
                     'album_name': tf.io.RaggedFeature(tf.string)
                    }
context_features = {"name" : tf.io.FixedLenFeature(dtype=tf.string, shape=(1)),
                    "collaborative" : tf.io.FixedLenFeature(dtype=tf.string, shape=(1)),
                    "modified_at" : tf.io.FixedLenFeature(dtype=tf.int64, shape=(1)),
                    "num_tracks" : tf.io.FixedLenFeature(dtype=tf.int64, shape=(1)),
                    "num_albums" : tf.io.FixedLenFeature(dtype=tf.int64, shape=(1)),
                    "num_followers" :tf.io.FixedLenFeature(dtype=tf.int64, shape=(1)),
                    "num_edits" :tf.io.FixedLenFeature(dtype=tf.int64, shape=(1)),
                    "duration_ms" : tf.io.FixedLenFeature(dtype=tf.int64, shape=(1)),
                    "num_artists" : tf.io.FixedLenFeature(dtype=tf.int64, shape=(1)),
                    "description" : tf.io.FixedLenFeature(dtype=tf.string, shape=(1))
                   }